In [ ]:
import requests
import pandas as pd

# Overpass API endpoint
overpass_url = "https://overpass-api.de/api/interpreter"

overpass_query = """
[out:json][timeout:50];
(
  node["amenity"="hospital"](27.55,85.20,27.85,85.55);
  way["amenity"="hospital"](27.55,85.20,27.85,85.55);
  relation["amenity"="hospital"](27.55,85.20,27.85,85.55);
);
out center;
"""

response = requests.post(overpass_url, data=overpass_query)
data = response.json()

hospitals = []
for element in data['elements']:
    tags = element.get("tags", {})
    
    hosp = {
        "id": element["id"],
        "type": element["type"],
        "name": tags.get("name", ""),
        "lat": element.get("lat", element.get("center", {}).get("lat", "")),
        "lon": element.get("lon", element.get("center", {}).get("lon", "")),
        "address": tags.get("addr:full", tags.get("addr:street", "")),
        "speciality": tags.get("healthcare:speciality", ""),
        "emergency": tags.get("emergency", ""),
        "icu": tags.get("facility:icu", ""),
        "operating_theatre": tags.get("facility:operating_theatre", ""),
        "ventilator": tags.get("facility:ventilator", ""),
        "x_ray": tags.get("facility:x-ray", ""),
        "opening_hours": tags.get("opening_hours", ""),
        "website": tags.get("website", ""),
        "phone": tags.get("phone", tags.get("contact:phone", "")),
    }
    
    hospitals.append(hosp)

# Convert to DataFrame
df = pd.DataFrame(hospitals)

# Save as CSV
df.to_csv("hospitals.csv", index=False)

print("CSV downloaded: hospitals.csv")


CSV downloaded: hospitals.csv


In [3]:
import pandas as pd
df=pd.read_csv("hospitals.csv")
print("Original Shape:", df.shape)
print("Columns:", df.columns.tolist())
if "type" in df.columns:
    df = df.drop(columns=["type"])
df = df.dropna(subset=["name"])

for col in ["emergency","icu", "operating_theatre", "ventilator", "x_ray"]:
    if col in df.columns:
        df[col] = df[col].fillna("yes")
if "opening_hours" in df.columns:
    df["opening_hours"] = df["opening_hours"].fillna("24/7")

df
df.to_csv("hospitals_modified.csv", index=False)
print("Modified data saved to 'hospitals_modified.csv'")


Original Shape: (301, 15)
Columns: ['id', 'type', 'name', 'lat', 'lon', 'address', 'speciality', 'emergency', 'icu', 'operating_theatre', 'ventilator', 'x_ray', 'opening_hours', 'website', 'phone']
Modified data saved to 'hospitals_modified.csv'
